In [2]:
import pandas as pd
import numpy as np
import json
import ast

***
## Proceso de ETL
***

### **E**xtraction

- La primera parte del proceso consiste en la extraccion de los datos desde su punto de origen, en nuestro caso se encuentran en un conjunto de datasets proporcionados por la empresa steam de tipo json.

In [3]:
games = 'datasets/output_steam_games.json'

# Se lee de cada línea del dataset
rows = []
with open(games) as f:
    for line in f.readlines():
        data = json.loads(line)
        rows.append(data)

# Se convierte a dataframe
games = pd.DataFrame(rows)
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [4]:
reviews = 'datasets/australian_user_reviews.json'

# Se lee de cada línea del dataset
rows.clear()
with open(reviews) as f:
    for line in f.readlines():
        data = ast.literal_eval(line)
        rows.append(data)

# Se convierte a dataframe
reviews = pd.DataFrame(rows)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [5]:
items = 'datasets/australian_users_items.json'

# Se lee de cada línea del dataset
rows.clear()
with open(items) as f:
    for line in f.readlines():
        data = ast.literal_eval(line)
        rows.append(data)

# Se convierte a dataframe
items = pd.DataFrame(rows)
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


***
### **T**ransform

- La fase de transformacion aplica una serie de reglas de negocio sobre los datos extraidos para poder limpiarlos y tener una buena calidad de estos al momento de cargarlos, la gravedad de estas transformaciones dependera de como se encuentren los datos extraidos. Segun nuestras tablas se requiere de una normalizacion en ellas y de algunos casting para mayor legibilidad y utilidad al momento de realizar la carga y otras operaciones.

#### *Resolver vacios*

In [6]:
print(f'Campos vacios en games: {games.isna().sum().sum()}')
print(f'Campos vacios en reviews: {reviews.isna().sum().sum()}')
print(f'Campos vacios en items: {items.isna().sum().sum()}')

Campos vacios en games: 1168997
Campos vacios en reviews: 0
Campos vacios en items: 0


In [7]:
#Resolvemos
games.dropna(inplace=True)
print(f'Vacios en games: {games.isna().sum().sum()}')

Vacios en games: 0


#### *Normalizacion de columnas*

#### *Games*

In [8]:
games.sample()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
97252,"CFK Co., Ltd.","[Casual, Indie]",Eraser & Builder,Eraser &amp; Builder,http://store.steampowered.com/app/583030/Erase...,2017-02-27,"[Casual, Indie]",http://steamcommunity.com/app/583030/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,583030,"CFK Co., Ltd."


In [9]:
print(f"Registro iguales entre la columna app_name y title: {(games['app_name']==games['title']).sum()}")
print(f"Cantidad de registros en el dataframe 'games': {len(games)}")

Registro iguales entre la columna app_name y title: 22179
Cantidad de registros en el dataframe 'games': 22530


La mayoria de los valores de una columna se encuentran en la otra, asi que dejaremos una de las dos para que sirva de referencia total al nombre del videojuego y aprovecharemos para eliminar columnas que no utilizaremos

In [10]:
games.drop(columns=['app_name','specs','early_access','publisher','tags','reviews_url'],inplace=True)

El dataframe posee una columna multivaluada que definitivamente nos conviene desglozar para un mejor uso

In [11]:
games = games.explode('genres')

Alguno valores en la columna price no son nisiquiera de tipo flotante, asi que arreglaremos eso cambiando esos valores y casteando la columna

In [12]:
games['price'] = pd.to_numeric(games['price'],errors='coerce').astype(float)
games['price'].fillna(0,inplace=True)

In [13]:
games[games['price']==0]

,genres,title,url,release_date,price,id,developer
88311,Free to Play,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,0.0,643980,Secret Level SRL
88311,Indie,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,0.0,643980,Secret Level SRL
88311,RPG,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,0.0,643980,Secret Level SRL
88311,Strategy,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,0.0,643980,Secret Level SRL
88312,Casual,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,0.0,670290,Poolians.com
...,...,...,...,...,...,...,...
120410,Free to Play,忍者村大战2,http://store.steampowered.com/app/754350/2/,2018-01-03,0.0,754350,杭州分浪网络科技有限公司
120410,Massively Multiplayer,忍者村大战2,http://store.steampowered.com/app/754350/2/,2018-01-03,0.0,754350,杭州分浪网络科技有限公司
120410,RPG,忍者村大战2,http://store.steampowered.com/app/754350/2/,2018-01-03,0.0,754350,杭州分浪网络科技有限公司
120410,Strategy,忍者村大战2,http://store.steampowered.com/app/754350/2/,2018-01-03,0.0,754350,杭州分浪网络科技有限公司


Ultimo retoque para un futuro merge con la tabla item que nos permita unificar las tres tablas

In [14]:
games.rename(columns={"title":"game_title"},inplace=True)

In [15]:
games['release_date'] = pd.to_datetime(games['release_date'],errors='coerce')
games.dropna(inplace=True)

In [16]:
games.sample()

,genres,game_title,url,release_date,price,id,developer
112196,Indie,Hyper Box,http://store.steampowered.com/app/438820/Hyper...,2016-03-16,2.99,438820,Pixelated Cupcake


#### *Reviews*

In [17]:
reviews.sample()

,user_id,user_url,reviews
7549,bountyassassin4,http://steamcommunity.com/id/bountyassassin4,"[{'funny': '', 'posted': 'Posted July 13, 2014..."


Otra columna multivaluada

In [18]:
reviews['reviews'].iloc[0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

In [19]:
# Desglosamos
reviews = reviews.explode('reviews')

In [20]:
# Dividimos cada llave de cada diccionario dentro de su propia columna dentro de nuestro dataset
reviews['posted'] = reviews['reviews'].apply(lambda x: np.nan if isinstance(x,float) else x['posted'])
reviews['item_id'] = reviews['reviews'].apply(lambda x: np.nan if isinstance(x,float) else x['item_id'])
reviews['recommend'] = reviews['reviews'].apply(lambda x: np.nan if isinstance(x,float) else x['recommend'])
reviews['review'] = reviews['reviews'].apply(lambda x: np.nan if isinstance(x,float) else x['review'])

In [21]:
print(f"Vacios: {reviews.isna().sum().sum()}")
reviews.dropna(inplace=True)
print(f"Vacios: {reviews.isna().sum().sum()}")

Vacios: 140
Vacios: 0


In [22]:
# Eliminamos la sobrante
reviews.drop(columns=['reviews'],inplace=True)

In [23]:
reviews.sample()

,user_id,user_url,posted,item_id,recommend,review
5162,greenkoala,http://steamcommunity.com/id/greenkoala,"Posted September 14, 2014.",249130,True,Has huge amounts of post game content. Loved i...


Las columnas de fechas deberian tener todas un mismo formato

In [24]:
# Dividimos las cadenas de texto en mes dia y año
reviews['posted'] = reviews['posted'].str.replace(',','').str.replace('.','').str.replace('Posted','').str.split()

# Creamos un hashmap para la traduccion de string a numero como referencia del mes 
months = {
  'January': 1,
  'February': 2,
  'March': 3,
  'April': 4,
  'May': 5,
  'June': 6,
  'July': 7,
  'August': 8,
  'September': 9,
  'October': 10,
  'November': 11,
  'December': 12
}
# Array para añadir cada registro ya traducido posteriormente al dataframe
dates=[]
for i,x in enumerate(reviews.itertuples()): # Se itera los registros

  if len(x.posted)==3: # Si el campo posee todo los campos encesarios para tener una fecha entonces se declara la fecha
    date = f"{x.posted[2]}-{months[x.posted[0]]}-{x.posted[1]}" 
  else: 
    date = np.nan

  dates.append(pd.to_datetime(date).date())

reviews['posted'] = pd.DataFrame(dates) # Se añaden a la columna del dataframe

#### *Feature Engineering*

Para una consulta en nuestra API vamos a necesitar una nueva columna que nos especifique cual es el sentimiento que emerge de las palabras de las reviews de los usuarios. Para eso utilizaremos una libreria famosa para el procesamiento de lenguaje natural que nos colaborara en el proceso. 

In [25]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

SID = SentimentIntensityAnalyzer()
analysis = []

for x in reviews.itertuples():
  sentiment = SID.polarity_scores(x.review)['compound']
  if sentiment<-.2:
    value = 0
  elif sentiment>.2:
    value = 2
  else:
    value = 1
  analysis.append(value)

reviews['sentiment'] = pd.DataFrame(analysis)

In [26]:
reviews.sample()

,user_id,user_url,posted,item_id,recommend,review,sentiment
21841,76561198067794298,http://steamcommunity.com/profiles/76561198067...,2014-10-04,253710,False,it really isnt that free to play,0


#### *Items*

In [27]:
items.sample()

,user_id,items_count,steam_id,user_url,items
72796,thejumpmaster3,11,76561198078987394,http://steamcommunity.com/id/thejumpmaster3,"[{'item_id': '313730', 'item_name': 'Sunrider:..."


Otra columna multivariada

In [28]:
# Se divide las listas por registro
items = items.explode('items')

In [29]:
# Dividimos cada llave de cada diccionario dentro de su propia columna dentro de nuestro dataset
items['item_id'] = items['items'].apply(lambda x: np.nan if isinstance(x,float) else x['item_id'])
items['item_name'] = items['items'].apply(lambda x: np.nan if isinstance(x,float) else x['item_name'])
items['playtime_forever'] = items['items'].apply(lambda x: np.nan if isinstance(x,float) else x['playtime_forever'])

In [30]:
# Validamos vacios
print(f"Vacios: {items.isna().sum().sum()}")
items.dropna(inplace=True)
print(f"Vacios: {items.isna().sum().sum()}")

Vacios: 67224
Vacios: 0


In [31]:
# Eliminamos columnas innecesarias
items.drop(columns=['items','steam_id','items_count','user_url'],inplace=True)

In [32]:
# Retoque final para el merge con la tabla games
items.rename(columns={"item_name":"game_title"},inplace=True)

In [33]:
items.sample()

,user_id,item_id,game_title,playtime_forever
59514,cordwil,43110,Metro 2033,0.0


***
### **L**oad
- Como cereza del pastel debemos cargar nuestros datos ya procesados a un sistema de origen. Este sistema dependera bastante de los requisitos de la organizacion y varia segun sus objetivos. Para este proyecto la carga se hara directamente en la Web mediante una API y en esta ocasion exportaremos solo parte de nuestro gran dataset, por cuestion de rendimiento, eficiencia y funcionamiento de la aplicacion que utilizaremos para el despliegue, Render. Para ser mas claros estos servicios poseen una prueba gratuita pero bastante limitada en cuestion de memoria.

In [56]:
game_item = pd.merge(games,items,on='game_title')
game_item.drop(columns=['game_title','url','id','item_id'],inplace=True)
game_item['release_date'] = pd.to_datetime(game_item['release_date']).dt.year

In [101]:
game_item.sample()

,genres,release_date,price,id,developer,user_id,playtime_forever
4950645,Free to Play,2014,0.0,242720,"Masangsoft, Inc.",76561198080248619,558.0


In [52]:
reviews_items = pd.merge(reviews,items[['item_id','game_title']].drop_duplicates(),on='item_id')
reviews_items.dropna(inplace=True)

In [58]:
all_dfs = pd.merge(reviews_items, games[['game_title','developer','id','url','price']].drop_duplicates(), on='game_title')
all_dfs.dropna(inplace=True)

In [59]:
all_dfs.sample()

,user_id,user_url,posted,item_id,recommend,review,sentiment,game_title,developer,id,url,price
8937,76561198123168383,http://steamcommunity.com/profiles/76561198123...,2015-12-25,221100,True,best game put in lots of hours and havent got ...,2,DayZ,Bohemia Interactive,221100,http://store.steampowered.com/app/221100/DayZ/,34.99


In [115]:
game_item[:len(game_item)//2].to_parquet('./clean_datasets/game_items.parquet')
all_dfs.to_parquet('./clean_datasets/all_dfs.parquet')

In [124]:
game_item[:len(game_item)//2]['developer'].drop_duplicates().sample()

1904003    Starfish-SD Inc
Name: developer, dtype: object

In [128]:
game_item[:len(game_item)//2][game_item[:len(game_item)//2]['price']==0]['developer'].drop_duplicates().sample()

2901886    Targem Games
Name: developer, dtype: object

In [190]:
def developer(developer:str):
  '''
  Devuelve la cantidad de items y el porcentaje de contenido Free por año
  segun empresa desarrolladora.
  '''
  df = game_item[game_item['developer']==developer]
  items = df.groupby('release_date')['developer'].count().reset_index()
  free = df[df['price']==0].groupby('release_date')['developer'].count().reset_index()
  
  
  rows = []
  merge = pd.merge(free,items,on='release_date')
  for x in items.itertuples():
    if x.release_date in list(merge['release_date']):
      free_content = round((merge.developer_x/merge.developer_y)*100,2).item()
    else:
      free_content = 0
    
    rows.append({"Año":x.release_date,"Cantidad de items":x.developer,"Contenido Free":f"{free_content}%"})
  res = {
    developer:rows
  }
  return res

In [191]:
developer('Targem Games')

{'Targem Games': [{'Año': 2003,
   'Cantidad de items': 44,
   'Contenido Free': '0%'},
  {'Año': 2005, 'Cantidad de items': 171, 'Contenido Free': '0%'},
  {'Año': 2006, 'Cantidad de items': 32, 'Contenido Free': '0%'},
  {'Año': 2007, 'Cantidad de items': 88, 'Contenido Free': '0%'},
  {'Año': 2008, 'Cantidad de items': 56, 'Contenido Free': '0%'},
  {'Año': 2009, 'Cantidad de items': 162, 'Contenido Free': '0%'},
  {'Año': 2012, 'Cantidad de items': 1154, 'Contenido Free': '0%'},
  {'Año': 2014, 'Cantidad de items': 528, 'Contenido Free': '0%'},
  {'Año': 2016, 'Cantidad de items': 3, 'Contenido Free': '0%'},
  {'Año': 2017, 'Cantidad de items': 72, 'Contenido Free': '100.0%'}]}

In [69]:
def user_data(user_id:str):
  '''
  Devuelve la cantidad de dinero gastado por el usuario mas el 
  porcentaje de recomendacion y la cantidad de items
  '''
  df = all_dfs[all_dfs['user_id']==user_id]
  money = df['price'].sum()
  items = len(df)

  user_reviews = df[df['user_id']==user_id]
  recommendation = (user_reviews['recommend'].sum()/len(user_reviews))*100

  res = {
    "Usuario X": user_id,
    "Dinero gastado": money,
    "% de recomendacion": f"{round(recommendation,2)}%",
    "Cantidad de items": items
  }

  return  res


In [70]:
user_data('54678907652')

{'Usuario X': '54678907652',
 'Dinero gastado': 89.94,
 '% de recomendacion': '42.86%',
 'Cantidad de items': 7}

In [96]:
def user_for_genre(genre:str):
  '''
  Devuelve el usuario con mas horas jugadas para el genero dado 
  y una lista de acumulaciones de horas jugadas por año
  '''
  df = game_item[game_item['genres']==genre]

  user = df.groupby('user_id')['playtime_forever'].sum().sort_values(ascending=False).index[0]
  hours = df[df['user_id']==user].groupby('release_date')['playtime_forever'].sum()

  year = []
  for i,x in enumerate(hours):
    if x!=0:
      year.append({"Año":type(hours.index[i].item()),"Horas":type(int(x))})

  res = {
    "Usuario con mas horas jugadas para Genero":type(user),
    "Horas Jugadas": type(year)
  }
  return res

In [97]:
user_for_genre('Adventure')

{'Usuario con mas horas jugadas para Genero Adventure': str,
 'Horas Jugadas': list}

In [98]:
def best_developer_year(year:int):
  '''
  Devuelve el top 3 de desarrolladores con juegos mas recomendados por usuarios para el año dado.
  '''
  df = all_dfs[pd.to_datetime(all_dfs['posted']).dt.year == int(year)].copy()
  df['posted'] = pd.to_datetime(df['posted']).dt.year
  recommendation = df.groupby('developer')['recommend'].sum().sort_values(ascending=False)[:3]
  
  res = {
    "Año":year,
    "Res":[
      {"Puesto 1":recommendation.index[0]},
      {"Puesto 2":recommendation.index[1]},
      {"Puesto 3":recommendation.index[2]}
    ]
  }
  return res

In [99]:
best_developer_year(2015)

{'Año': 2015, 'Res': list}

In [48]:

def developer_reviews_analysis(developer:str):
  '''
  Devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un analisis de sentimiento como valor positivo o negativo.
  '''
  sentiments = all_dfs['sentiment'].value_counts()
  res = {
    developer:{
      "Negative":sentiments.values[2], 
      "Positive":sentiments.values[0]
    }
  }
  return res

In [49]:
developer_reviews_analysis('Valve')

{'Valve': {'Negative': 4221, 'Positive': 17638}}